In [1]:
!whoami

root


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,avg
import time

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("MySparkApp") \
    .master("spark://master:7077") \
    .config("spark.jars.packages", "net.snowflake:spark-snowflake_2.12:2.16.0-spark_3.4")  \
    .getOrCreate()

spark

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
net.snowflake#spark-snowflake_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2940c246-f79b-43ec-8b20-6548a0959243;1.0
	confs: [default]
	found net.snowflake#spark-snowflake_2.12;2.16.0-spark_3.4 in central
	found net.snowflake#snowflake-jdbc;3.16.1 in central
downloading https://repo1.maven.org/maven2/net/snowflake/spark-snowflake_2.12/2.16.0-spark_3.4/spark-snowflake_2.12-2.16.0-spark_3.4.jar ...
	[SUCCESSFUL ] net.snowflake#spark-snowflake_2.12;2.16.0-spark_3.4!spark-snowflake_2.12.jar (1369ms)
downloading https://repo1.maven.org/maven2/net/snowflake/snowflake-jdbc/3.16.1/snowflake-jdbc-3.16.1.jar ...
	[SUCCESSFUL ] net.snowflake#snowflake-jdbc;3.16.1!snowflake-jdbc.jar (5674ms)
:: resolution report :: resolve 2558ms :: artifacts dl 7052ms
	:: modules in use:
	net.snowflake#snowflake-jdbc;3.16.1 from central in [default]
	net.snowflake#spark-

In [3]:
for i in spark.sparkContext.getConf().getAll(): print(i)

('spark.repl.local.jars', 'file:///root/.ivy2/jars/net.snowflake_spark-snowflake_2.12-2.16.0-spark_3.4.jar,file:///root/.ivy2/jars/net.snowflake_snowflake-jdbc-3.16.1.jar')
('spark.driver.extraJavaOptions', '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.secu

In [3]:
# Create a sample DataFrame
data = [("Alice", 34), ("Bob", 45), ("Cathy", 29), ("David", 35)]
columns = ["Name", "Age"]

df = spark.createDataFrame(data, columns)

# Show the DataFrame
df.show()

# Add a new column with age incremented by 1
df_with_new_col = df.withColumn("AgePlusOne", col("Age") + 1)

# Show the DataFrame with the new column
df_with_new_col.show()

# Calculate the average age
average_age = df.agg(avg("Age")).collect()[0][0]
print(f"The average age is: {average_age}")

# Filter the DataFrame to include only rows where age is greater than 30
filtered_df = df.filter(col("Age") > 30)

# Show the filtered DataFrame
filtered_df.show()

+-----+---+
| Name|Age|
+-----+---+
|Alice| 34|
|  Bob| 45|
|Cathy| 29|
|David| 35|
+-----+---+

+-----+---+----------+
| Name|Age|AgePlusOne|
+-----+---+----------+
|Alice| 34|        35|
|  Bob| 45|        46|
|Cathy| 29|        30|
|David| 35|        36|
+-----+---+----------+



The average age is: 35.75
+-----+---+
| Name|Age|
+-----+---+
|Alice| 34|
|  Bob| 45|
|David| 35|
+-----+---+



In [6]:
df = spark.read.csv("file:///data/Player.csv", header=True)
df.show()


+---+-------------+--------------------+------------------+-------------------+------+------+
| id|player_api_id|         player_name|player_fifa_api_id|           birthday|height|weight|
+---+-------------+--------------------+------------------+-------------------+------+------+
|  1|       505942|  Aaron Appindangoye|            218353|1992-02-29 00:00:00|182.88|   187|
|  2|       155782|     Aaron Cresswell|            189615|1989-12-15 00:00:00|170.18|   146|
|  3|       162549|         Aaron Doran|            186170|1991-05-13 00:00:00|170.18|   163|
|  4|        30572|       Aaron Galindo|            140161|1982-05-08 00:00:00|182.88|   198|
|  5|        23780|        Aaron Hughes|             17725|1979-11-08 00:00:00|182.88|   154|
|  6|        27316|          Aaron Hunt|            158138|1986-09-04 00:00:00|182.88|   161|
|  7|       564793|          Aaron Kuhl|            221280|1996-01-30 00:00:00|172.72|   146|
|  8|        30895|        Aaron Lennon|            152747|1

In [8]:
# Snowflake connection options
sfOptions = {
    "sfURL": f"https://as37355.ap-south-1.aws.snowflakecomputing.com",
    "sfUser": "srinitheka",
    "sfPassword": "Cottoncandy@123",
    "sfDatabase": "TEST",
    "sfSchema": "TEST",
    "sfWarehouse": "compute_wh",
    "sfRole": "ACCOUNTADMIN"
}

# Read the data from Snowflake table into a DataFrame
df = spark.read \
    .format("net.snowflake.spark.snowflake") \
    .options(**sfOptions) \
    .option("dbtable", "TEST.TEST.CREDIT_CARD") \
    .load()

# Register the DataFrame as a temporary SQL view for processing
df.createOrReplaceTempView("CREDIT_CARD")
df.show()

24/08/17 15:03:46 WARN SnowflakeConnectorUtils$: Query pushdown is not supported because you are using Spark 3.5.2 with a connector designed to support Spark 3.4. Either use the version of Spark supported by the connector or install a version of the connector that supports your version of Spark.
24/08/17 15:03:49 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/08/17 15:03:55 ERROR Inbox: Ignoring error                       (0 + 1) / 1]
java.io.NotSerializableException: org.apache.spark.storage.StorageStatus
Serialization stack:
	- object not serializable (class: org.apache.spark.storage.StorageStatus, value: org.apache.spark.storage.StorageStatus@6ee00915)
	- element of array (index: 0)
	- array (class [Lorg.apache.spark.storage.StorageStatus;, size 3)
	at org.apache.spark.serializer.SerializationDebugger$.improveException(SerializationDebugger.scala:41)
	at or

+------+-------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+------------------+-------------------+-------------------+------------------+-------------------+-------------------+------------------+------------------+-------------------+-------------------+--------------------+------------------+-------------------+-------------------+-------------------+------------------+--------------------+-------------------+------+-----+
|  TIME|                 V1|                 V2|                V3|                V4|                 V5|                 V6|                 V7|                  V8|                 V9|                V10|               V11|                V12|                V13|               V14|                V15|                V16|               V17|               V18|                V19|                V20|                 V21

24/08/17 15:11:44 WARN SparkConnectorContext$: Finish cancelling all queries for app-20240817150128-0001
